In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [4]:
access_path = './data/processed/'
# bal_from_nm = 'bal_btw_rebal_20210602.dat'
# last_rebal_nm = 'balance_rebaldate_0702_all.dat'
# last_rebal_nm = 'balance_rebaldate_0801_all.dat'
# last_rebal_nm = 'balance_rebaldate_0901_all.dat'
last_rebal_nm = 'balance_rebaldate_0930_all.dat'

In [5]:
bal_from = pd.read_csv(access_path + last_rebal_nm)
price_db = PriceDB.instance().data

In [6]:
price_db

,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2018-07-20,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
1,2018-07-21,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
2,2018-07-22,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
3,2018-07-23,A069500,27685.0000,7283838.0,219684.583805,-0.008094,ETF
4,2018-07-24,A069500,27770.0000,2562621.0,170177.717593,0.003066,ETF
...,...,...,...,...,...,...,...
40642,2021-10-02,CALL,0.0071,NaN,NaN,NaN,riskfree
40643,2021-10-03,CALL,0.0071,NaN,NaN,NaN,riskfree
40644,2021-10-04,CALL,0.0071,NaN,NaN,NaN,riskfree
40645,2021-10-05,CALL,0.0069,NaN,NaN,NaN,riskfree


#### MP 발생일
- 2021.5.3
- 2021.6.2
- 2021.7.2
- 2021.8.1 (일요일)
- 2021.9.1
- 2021.10.1
- 2021.10.7 (임시)

In [7]:
# 두 MP발생일 사이 기간 설정
start_dt = '20211002'  # MP발생 다음날부터
end_dt = '20211006'    # MP발생 전날까지

range_dt_ymd = pd.date_range(start=start_dt, end=end_dt, freq='D')  # price_db 날짜 포맷
range_dt_mdy = range_dt_ymd.strftime('%-m/%-d/%Y')  # 잔고(balance) 날짜 포맷

In [8]:
# price_db 마지막 날짜를 휴일에 복사하기
# temp = price_db[price_db.date==price_db.iloc[-1,0]].copy()
# temp.date = '2021-07-31'
# price_db = pd.concat((price_db, temp), axis=0)
# price_db = price_db.reset_index(drop=True)

#### 최근 과거 MP발생일 잔고를 해당 기간 내 매일 매일의 잔고로 복사해 두고, price 컬럼과 value 컬럼을 업데이트하면 끝.

- 잔고 복사

In [9]:
df_list = []
for cnt, dt in enumerate(range_dt_ymd):
    temp = bal_from.copy(deep=True)
    temp.date = dt
    df_list.append(temp)
    
bal_btw_rebal = pd.concat(df_list)

- `price_db`를 이용해서 Price 컬럼 업데이트.

In [10]:
bal_btw_rebal = bal_btw_rebal.set_index(['date', 'itemcode'])
price_db = price_db.set_index(['date', 'itemcode'])
bal_btw_rebal.update(price_db)
bal_btw_rebal = bal_btw_rebal.reset_index(drop=False)

- 사용자별/날짜별로 비중(`wt`) 계산

In [11]:
bal_btw_rebal.value = bal_btw_rebal.quantity*bal_btw_rebal.price
bal_btw_rebal.original = ''
bal_btw_rebal.wt = bal_btw_rebal.value/bal_btw_rebal.groupby(['date', 'userid'])['value'].transform('sum')

In [12]:
bal_btw_rebal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-10-02,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,57760.0,115520.0,57675.0,115350.0,0.162528,NaN,
1,2021-10-02,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101265.0,101265.0,101265.0,101265.0,0.142682,NaN,
2,2021-10-02,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101245.0,202490.0,101245.0,202490.0,0.285308,NaN,
3,2021-10-02,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104605.0,104605.0,104610.0,104610.0,0.147395,NaN,
4,2021-10-02,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,108380.0,108380.0,108350.0,108350.0,0.152665,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2021-10-06,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23.0,12875.0,296125.0,12230.0,281290.0,0.230414,NaN,
356,2021-10-06,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10075.0,171275.0,9580.0,162860.0,0.133404,NaN,
357,2021-10-06,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,40195.0,120585.0,38260.0,114780.0,0.094020,NaN,
358,2021-10-06,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,57760.0,57760.0,57515.0,57515.0,0.047112,NaN,


In [13]:
bal_btw_rebal.to_csv(access_path + 'bal_btw_rebal.dat', index=False, encoding='utf-8-sig')

In [14]:
bal_btw_rebal.columns

Index(['date', 'itemcode', 'userid', 'name', 'asset_class', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'original'],
      dtype='object')